In [ ]:
import numpy as np
import pandas as pd
import skimage
import skimage.io
import skimage.filters
import skimage.morphology
from scipy import ndimage as ndi
from skimage.filters import threshold_otsu

from skimage.morphology import watershed
from skimage.feature import peak_local_max
from skimage.measure import label
from skimage.morphology import closing, square
from skimage.measure import regionprops
from skimage.color import label2rgb

import os
import glob

import bebi103

import colorcet

import bokeh
bokeh.io.output_notebook()

import holoviews as hv
hv.extension('bokeh')
bebi103.hv.set_defaults()

In [ ]:
# The directory containing the images
data_dir = '..\\data\\barcode'

# glob string for images
im_glob = os.path.join(data_dir, 'Round1_max_composite-25-6.tif')

# Get list of images
im_list = sorted(glob.glob(im_glob))

im_list

In [ ]:
# Read data using skimage
im = skimage.io.imread(im_list[0])

In [ ]:
dapi = im
p = bebi103.image.imshow(dapi)
bokeh.io.show(p)

I will define two functions to display two or three plots side by side.

In [ ]:
def show_two_ims(
    im_1,
    im_2,
    titles=[None, None],
    interpixel_distances=[0.13, 0.13],
    cmap=[None, None]
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=225,
        title=titles[0],
        cmap=cmap[0],
        #interpixel_distance=interpixel_distances[0],
        #length_units="µm",
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=225,
        title=titles[1],
        cmap=cmap[1],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range

    return bokeh.layouts.gridplot([p_1, p_2], ncols=2)

def show_three_ims(
    im_1,
    im_2,
    im_3,
    titles=[None, None, None],
    interpixel_distances=[0.13, 0.13, 0.13],
    cmap=[None, None, None],
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=225,
        title=titles[0],
        cmap=cmap[0],
        #interpixel_distance=interpixel_distances[0],
        #length_units="µm",
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=225,
        title=titles[1],
        cmap=cmap[1],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_3 = bebi103.image.imshow(
        im_3,
        frame_height=225,
        title=titles[2],
        cmap=cmap[2],
        #interpixel_distance=interpixel_distances[1],
        #length_units="µm",
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range
    p_3.x_range = p_1.x_range
    p_3.y_range = p_1.y_range
    
    return bokeh.layouts.gridplot([p_1, p_2, p_3], ncols=3)

## Step 1. Filters

First, I will apply a gaussian filter.

In [ ]:
# convert image to float
dapi_float = skimage.img_as_float(dapi)

# Make slice object
zoom1 = np.s_[750:1250, 775:1275]

In [ ]:
# Filter image w/ gaussian 
dapi_filt_gauss = skimage.filters.gaussian(dapi_float, 2)

# Show filtered image
bokeh.io.show(
    show_two_ims(dapi_float[zoom1], dapi_filt_gauss[zoom1], 
    titles=["original", "gaussian filtered"]))

## Step 2. Thresholding

I will threshold using Otsu's method.

In [ ]:
def plot_hist(im, title, logy=False):
    """Make plot of image histogram."""
    counts, vals = skimage.exposure.histogram(im)
    if logy:
        inds = counts > 0
        log_counts = np.log(counts[inds])
        return hv.Spikes(
            data=(vals[inds], log_counts),
            kdims=['pixel values'],
            vdims=['log₁₀ count'],
            label=title,
        ).opts(
            frame_height=100,
        )

    return hv.Spikes(
        data=(vals, counts),
        kdims=['pixel values'],
        vdims=['count'],
        label=title,
    ).opts(
        frame_height=100,
    )

In [ ]:
# Display histograms
plots = [plot_hist(dapi_float, 'no filter'),
         plot_hist(dapi_filt_gauss, 'gaussian filter')
        ]
hv.Layout(
    plots
).opts(
    shared_axes=False,
).cols(
    1
)

From this histogram, it seems like near 0.0017 is where I should define my cutoff point. I will use Otsu's thresholding method to define this point exactly. Then, I will threshold the original image, the gaussian filtered image, and the total variation filtered image.

In [ ]:
threshold = threshold_otsu(dapi_float)
print(threshold, 'is where the cutoff point is.')
dapi_bw = dapi_float > threshold
dapi_filt_gauss_bw = dapi_filt_gauss > threshold

# Show images
bokeh.io.show(
    show_two_ims(dapi_bw[zoom1], dapi_filt_gauss_bw[zoom1],
                   titles=['original', 'gauss filter']))


Now, I will dilate the image to make the nuclei more round.

In [ ]:
# Make the structuring element 1 pixel radius disk
selem = skimage.morphology.disk(1)

# Dilate image
dapi_bw_dil = skimage.morphology.dilation(dapi_bw, selem)
dapi_filt_gauss_bw_dil = skimage.morphology.dilation(dapi_filt_gauss_bw, selem)

# Show images
bokeh.io.show(
    show_two_ims(dapi_bw_dil[zoom1], dapi_filt_gauss_bw_dil[zoom1], 
                   titles=['dil original', 'dil gauss filter']))

In [ ]:
zoom2 = np.s_[1500:2000, 775:1275]

# Show images
bokeh.io.show(
    show_two_ims(dapi_bw_dil[zoom2], dapi_filt_gauss_bw_dil[zoom2],
                   titles=['dil original', 'dil gauss filter']))

It seems like the dilation process has merged some nuclei. Let's try to use the watershed tool to separate the merged nuclei. I will do this by performing distance transformation, followed by identification of local maxima. I will then use these maxima as markers, for which I will perform the watershed. The watershed will be performed on the mask from the thresholded gaussian filter image.

In [ ]:
distance = ndi.distance_transform_edt(dapi_filt_gauss_bw_dil)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((150, 150)),
                            labels=dapi_filt_gauss_bw_dil)
markers = ndi.label(local_maxi)[0]
labels = watershed(-distance, markers, mask=dapi_filt_gauss_bw_dil)
# bokeh.io.show(show_three_ims(distance[zoom2], markers[zoom2], labels[zoom2], 
#                              titles=['distance', 'local maxi', 'watershed'], 
#                              cmap=[None, colorcet.gray, colorcet.b_glasbey_hv]))

dapi_ws = skimage.segmentation.clear_border(labels, buffer_size=5)
dapi_ws_mask = skimage.morphology.remove_small_objects(dapi_ws, min_size=5000)
bokeh.io.show(
    show_three_ims(dapi, dapi_ws, dapi_ws_mask,
                titles=['original', 'watershed', 'watershed remove small'],
                cmap=[None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

Amjad: instead of using "peak_local_max", I used H-minima transform to get rid of insignificant minima. The results for this test image is pretty similar to yours. But it may be more general, in cases where nuclei are of different sizes.

In [ ]:
distance = -1 * ndi.distance_transform_edt(dapi_filt_gauss_bw_dil)
local_mini = skimage.morphology.h_minima(distance, 10)
mrkr = ndi.label(local_mini)[0]
labels = skimage.morphology.watershed(distance, markers = mrkr,\
                                      mask=dapi_filt_gauss_bw_dil, watershed_line=True)
# bokeh.io.show(show_three_ims(distance[zoom2], markers[zoom2], labels[zoom2], 
#                              titles=['distance', 'local maxi', 'watershed'], 
#                              cmap=[None, colorcet.gray, colorcet.b_glasbey_hv]))

dapi_ws = skimage.segmentation.clear_border(labels, buffer_size=5)
dapi_ws_mask = skimage.morphology.remove_small_objects(dapi_ws, min_size=5000)
bokeh.io.show(
    show_three_ims(dapi, dapi_ws, dapi_ws_mask,
                titles=['original', 'watershed', 'watershed remove small'],
                cmap=[None, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

In [ ]:
# relabel image regions
label_image = label(dapi_ws_mask)
bokeh.io.show(bebi103.image.imshow(label_image, cmap=colorcet.b_glasbey_hv))

In [ ]:
props = skimage.measure.regionprops_table(label_image, intensity_image=dapi, properties=('label',
                                                                                         'centroid',
                                                                                         'area',
                                                                                         'mean_intensity'))
df = pd.DataFrame(props)
df